In [13]:
import os
import random

import numpy as np
import torch as t
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn

class Config():

    def __init__(self):

        self.device = 'cpu'
        self.seed = 0

        self.datasets_dir = ''

        self.n_epoch = 100
        self.batch_size = 32
        self.n_worker = 0
        self.lr = 1.0

        self.ckpts_dir = ''

        return

class Task(Dataset):
    def __init__(self, cfg):
        self.data_ls = []
        with open(os.path.join('data/', 'train.csv'), 'r') as f:
            i=0
            for line in f.readlines():
                if i>0:
                    value_ls = line.strip('\n').split(',')
                    value_ls[3] = value_ls[3][9]
                    value_ls[4] = value_ls[4][9]
                    if value_ls[1] == 'A':
                        value_ls[1] = 0.0
                    elif value_ls[1] == 'B':
                        value_ls[1] = 1.0
                    elif value_ls[1] == 'C':
                        value_ls[1] = 2.0
                    elif value_ls[1] == 'D':
                        value_ls[1] = 3.0
                    elif value_ls[1] == 'E':
                        value_ls[1] = 4.0
                    for j in range(1,len(value_ls)):
                        if value_ls[j] == '':
                            value_ls[j] = 0.0
                        else:
                            value_ls[j] = float(value_ls[j])
                    value_ls.pop(0)        
                    arr = np.array(value_ls)
                    arr = arr.astype(np.single)
                    self.data_ls.append(arr)
                i=i+1
        self.data = np.stack(self.data_ls,axis=0)
        return
    
    def __getitem__(self, idx):
        x = self.data[idx, :-1]
#         print(x)
        y = self.data[idx, -1]
        return x, y
        
    def __len__(self):
        return self.data.shape[0]
    
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.C0 = nn.Sequential(
            nn.Linear(24, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Linear(100,256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64,1)
            )
        return
    
    def forward(self,x):
        pred_y = self.C0(x)
        return pred_y
    
cfg = Config()  
cfg.ckpts_dir = 'ckpts/'

random.seed(cfg.seed)
np.random.seed(cfg.seed)
t.manual_seed(cfg.seed)
t.backends.cudnn.deterministic = True
t.backends.cudnn.benchmark = False

train_dataset = Task(cfg)
train_dataloader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.n_worker)

model = Model()
model = model.to(cfg.device)

optimizer = t.optim.SGD(model.parameters(), lr=cfg.lr)

if not os.path.isdir(cfg.ckpts_dir):
    os.makedirs(cfg.ckpts_dir)

for epoch in range(cfg.n_epoch):
    
    for i,(x0,y0) in enumerate(train_dataloader):
        
        x0 = x0.to(cfg.device)
        y0 = y0.to(cfg.device)
        y0_ = model(x0)
        loss = nn.BCEWithLogitsLoss()
        y0 = y0.view(-1, 1)
        output = loss(y0_, y0)
        
        optimizer.zero_grad()
        output.backward()
        optimizer.step()
        
        print(f'Epoch: {epoch}, Iter: {i}, Loss: {output.item():.5f}')
        
    t.save(model.state_dict(), os.path.join(cfg.ckpts_dir, f'model_e{epoch}.ckpt'))    

Epoch: 0, Iter: 0, Loss: 0.68256
Epoch: 0, Iter: 1, Loss: 0.75151
Epoch: 0, Iter: 2, Loss: 0.90007
Epoch: 0, Iter: 3, Loss: 1.26789
Epoch: 0, Iter: 4, Loss: 0.58265
Epoch: 0, Iter: 5, Loss: 0.37135
Epoch: 0, Iter: 6, Loss: 0.62312
Epoch: 0, Iter: 7, Loss: 0.44718
Epoch: 0, Iter: 8, Loss: 0.61336
Epoch: 0, Iter: 9, Loss: 0.47552
Epoch: 0, Iter: 10, Loss: 0.61097
Epoch: 0, Iter: 11, Loss: 0.52343
Epoch: 0, Iter: 12, Loss: 0.70071
Epoch: 0, Iter: 13, Loss: 0.91634
Epoch: 0, Iter: 14, Loss: 0.67749
Epoch: 0, Iter: 15, Loss: 0.80032
Epoch: 0, Iter: 16, Loss: 0.55893
Epoch: 0, Iter: 17, Loss: 0.50644
Epoch: 0, Iter: 18, Loss: 0.28028
Epoch: 0, Iter: 19, Loss: 0.50428
Epoch: 0, Iter: 20, Loss: 0.69035
Epoch: 0, Iter: 21, Loss: 0.66519
Epoch: 0, Iter: 22, Loss: 0.56961
Epoch: 0, Iter: 23, Loss: 0.59695
Epoch: 0, Iter: 24, Loss: 0.55655
Epoch: 0, Iter: 25, Loss: 0.43763
Epoch: 0, Iter: 26, Loss: 0.84379
Epoch: 0, Iter: 27, Loss: 0.66053
Epoch: 0, Iter: 28, Loss: 0.58946
Epoch: 0, Iter: 29, Loss

Epoch: 0, Iter: 260, Loss: 0.45115
Epoch: 0, Iter: 261, Loss: 0.46394
Epoch: 0, Iter: 262, Loss: 0.38199
Epoch: 0, Iter: 263, Loss: 0.52914
Epoch: 0, Iter: 264, Loss: 0.55962
Epoch: 0, Iter: 265, Loss: 0.50426
Epoch: 0, Iter: 266, Loss: 0.61420
Epoch: 0, Iter: 267, Loss: 0.39615
Epoch: 0, Iter: 268, Loss: 0.39732
Epoch: 0, Iter: 269, Loss: 0.62371
Epoch: 0, Iter: 270, Loss: 0.47801
Epoch: 0, Iter: 271, Loss: 0.50110
Epoch: 0, Iter: 272, Loss: 0.52166
Epoch: 0, Iter: 273, Loss: 0.46549
Epoch: 0, Iter: 274, Loss: 0.48357
Epoch: 0, Iter: 275, Loss: 0.66400
Epoch: 0, Iter: 276, Loss: 0.59372
Epoch: 0, Iter: 277, Loss: 0.61603
Epoch: 0, Iter: 278, Loss: 0.39386
Epoch: 0, Iter: 279, Loss: 0.47139
Epoch: 0, Iter: 280, Loss: 0.53664
Epoch: 0, Iter: 281, Loss: 0.44724
Epoch: 0, Iter: 282, Loss: 0.55493
Epoch: 0, Iter: 283, Loss: 0.54299
Epoch: 0, Iter: 284, Loss: 0.37878
Epoch: 0, Iter: 285, Loss: 0.43215
Epoch: 0, Iter: 286, Loss: 0.42913
Epoch: 0, Iter: 287, Loss: 0.61627
Epoch: 0, Iter: 288,

Epoch: 0, Iter: 531, Loss: 0.56223
Epoch: 0, Iter: 532, Loss: 0.66208
Epoch: 0, Iter: 533, Loss: 0.59866
Epoch: 0, Iter: 534, Loss: 0.51635
Epoch: 0, Iter: 535, Loss: 0.42550
Epoch: 0, Iter: 536, Loss: 0.43242
Epoch: 0, Iter: 537, Loss: 0.35161
Epoch: 0, Iter: 538, Loss: 0.67521
Epoch: 0, Iter: 539, Loss: 0.41123
Epoch: 0, Iter: 540, Loss: 0.52530
Epoch: 0, Iter: 541, Loss: 0.37184
Epoch: 0, Iter: 542, Loss: 0.44425
Epoch: 0, Iter: 543, Loss: 0.32891
Epoch: 0, Iter: 544, Loss: 0.64244
Epoch: 0, Iter: 545, Loss: 0.44662
Epoch: 0, Iter: 546, Loss: 0.59297
Epoch: 0, Iter: 547, Loss: 0.42737
Epoch: 0, Iter: 548, Loss: 0.45917
Epoch: 0, Iter: 549, Loss: 0.43142
Epoch: 0, Iter: 550, Loss: 0.42790
Epoch: 0, Iter: 551, Loss: 0.33256
Epoch: 0, Iter: 552, Loss: 0.49899
Epoch: 0, Iter: 553, Loss: 0.55225
Epoch: 0, Iter: 554, Loss: 0.43270
Epoch: 0, Iter: 555, Loss: 0.49480
Epoch: 0, Iter: 556, Loss: 0.53028
Epoch: 0, Iter: 557, Loss: 0.33479
Epoch: 0, Iter: 558, Loss: 0.64004
Epoch: 0, Iter: 559,

Epoch: 0, Iter: 768, Loss: 0.65184
Epoch: 0, Iter: 769, Loss: 0.54966
Epoch: 0, Iter: 770, Loss: 0.56548
Epoch: 0, Iter: 771, Loss: 0.44897
Epoch: 0, Iter: 772, Loss: 0.42750
Epoch: 0, Iter: 773, Loss: 0.52556
Epoch: 0, Iter: 774, Loss: 0.50430
Epoch: 0, Iter: 775, Loss: 0.55621
Epoch: 0, Iter: 776, Loss: 0.28807
Epoch: 0, Iter: 777, Loss: 0.44908
Epoch: 0, Iter: 778, Loss: 0.74699
Epoch: 0, Iter: 779, Loss: 0.53042
Epoch: 0, Iter: 780, Loss: 0.48529
Epoch: 0, Iter: 781, Loss: 0.49331
Epoch: 0, Iter: 782, Loss: 0.49507
Epoch: 0, Iter: 783, Loss: 0.35353
Epoch: 0, Iter: 784, Loss: 0.48648
Epoch: 0, Iter: 785, Loss: 0.55312
Epoch: 0, Iter: 786, Loss: 0.65065
Epoch: 0, Iter: 787, Loss: 0.47964
Epoch: 0, Iter: 788, Loss: 0.56298
Epoch: 0, Iter: 789, Loss: 0.50753
Epoch: 0, Iter: 790, Loss: 0.49514
Epoch: 0, Iter: 791, Loss: 0.72345
Epoch: 0, Iter: 792, Loss: 0.54856
Epoch: 0, Iter: 793, Loss: 0.45469
Epoch: 0, Iter: 794, Loss: 0.42391
Epoch: 0, Iter: 795, Loss: 0.55854
Epoch: 0, Iter: 796,

Epoch: 1, Iter: 215, Loss: 0.58860
Epoch: 1, Iter: 216, Loss: 0.49861
Epoch: 1, Iter: 217, Loss: 0.47122
Epoch: 1, Iter: 218, Loss: 0.42415
Epoch: 1, Iter: 219, Loss: 0.51432
Epoch: 1, Iter: 220, Loss: 0.56651
Epoch: 1, Iter: 221, Loss: 0.42545
Epoch: 1, Iter: 222, Loss: 0.52375
Epoch: 1, Iter: 223, Loss: 0.54038
Epoch: 1, Iter: 224, Loss: 0.55280
Epoch: 1, Iter: 225, Loss: 0.51734
Epoch: 1, Iter: 226, Loss: 0.61335
Epoch: 1, Iter: 227, Loss: 0.48238
Epoch: 1, Iter: 228, Loss: 0.54703
Epoch: 1, Iter: 229, Loss: 0.66517
Epoch: 1, Iter: 230, Loss: 0.49158
Epoch: 1, Iter: 231, Loss: 0.45026
Epoch: 1, Iter: 232, Loss: 0.68095
Epoch: 1, Iter: 233, Loss: 0.51341
Epoch: 1, Iter: 234, Loss: 0.50552
Epoch: 1, Iter: 235, Loss: 0.60483
Epoch: 1, Iter: 236, Loss: 0.34556
Epoch: 1, Iter: 237, Loss: 0.63497
Epoch: 1, Iter: 238, Loss: 0.51735
Epoch: 1, Iter: 239, Loss: 0.54107
Epoch: 1, Iter: 240, Loss: 0.47007
Epoch: 1, Iter: 241, Loss: 0.51107
Epoch: 1, Iter: 242, Loss: 0.48419
Epoch: 1, Iter: 243,

...

Epoch: 98, Iter: 646, Loss: 0.41444
Epoch: 98, Iter: 647, Loss: 0.55039
Epoch: 98, Iter: 648, Loss: 0.58128
Epoch: 98, Iter: 649, Loss: 0.52473
Epoch: 98, Iter: 650, Loss: 0.49599
Epoch: 98, Iter: 651, Loss: 0.66453
Epoch: 98, Iter: 652, Loss: 0.62772
Epoch: 98, Iter: 653, Loss: 0.56835
Epoch: 98, Iter: 654, Loss: 0.43637
Epoch: 98, Iter: 655, Loss: 0.48629
Epoch: 98, Iter: 656, Loss: 0.38739
Epoch: 98, Iter: 657, Loss: 0.42861
Epoch: 98, Iter: 658, Loss: 0.66814
Epoch: 98, Iter: 659, Loss: 0.61063
Epoch: 98, Iter: 660, Loss: 0.55801
Epoch: 98, Iter: 661, Loss: 0.44410
Epoch: 98, Iter: 662, Loss: 0.74035
Epoch: 98, Iter: 663, Loss: 0.39195
Epoch: 98, Iter: 664, Loss: 0.52533
Epoch: 98, Iter: 665, Loss: 0.52164
Epoch: 98, Iter: 666, Loss: 0.59298
Epoch: 98, Iter: 667, Loss: 0.47883
Epoch: 98, Iter: 668, Loss: 0.43902
Epoch: 98, Iter: 669, Loss: 0.49694
Epoch: 98, Iter: 670, Loss: 0.47692
Epoch: 98, Iter: 671, Loss: 0.62508
Epoch: 98, Iter: 672, Loss: 0.40834
Epoch: 98, Iter: 673, Loss: 

Epoch: 99, Iter: 62, Loss: 0.43468
Epoch: 99, Iter: 63, Loss: 0.53166
Epoch: 99, Iter: 64, Loss: 0.69395
Epoch: 99, Iter: 65, Loss: 0.44162
Epoch: 99, Iter: 66, Loss: 0.53405
Epoch: 99, Iter: 67, Loss: 0.68604
Epoch: 99, Iter: 68, Loss: 0.40363
Epoch: 99, Iter: 69, Loss: 0.66372
Epoch: 99, Iter: 70, Loss: 0.55610
Epoch: 99, Iter: 71, Loss: 0.54971
Epoch: 99, Iter: 72, Loss: 0.60551
Epoch: 99, Iter: 73, Loss: 0.49969
Epoch: 99, Iter: 74, Loss: 0.52000
Epoch: 99, Iter: 75, Loss: 0.33231
Epoch: 99, Iter: 76, Loss: 0.65814
Epoch: 99, Iter: 77, Loss: 0.46086
Epoch: 99, Iter: 78, Loss: 0.54481
Epoch: 99, Iter: 79, Loss: 0.47495
Epoch: 99, Iter: 80, Loss: 0.28682
Epoch: 99, Iter: 81, Loss: 0.46042
Epoch: 99, Iter: 82, Loss: 0.42298
Epoch: 99, Iter: 83, Loss: 0.48364
Epoch: 99, Iter: 84, Loss: 0.52623
Epoch: 99, Iter: 85, Loss: 0.45347
Epoch: 99, Iter: 86, Loss: 0.56672
Epoch: 99, Iter: 87, Loss: 0.37250
Epoch: 99, Iter: 88, Loss: 0.54663
Epoch: 99, Iter: 89, Loss: 0.52638
Epoch: 99, Iter: 90,

Epoch: 99, Iter: 310, Loss: 0.50801
Epoch: 99, Iter: 311, Loss: 0.55010
Epoch: 99, Iter: 312, Loss: 0.60243
Epoch: 99, Iter: 313, Loss: 0.36865
Epoch: 99, Iter: 314, Loss: 0.58516
Epoch: 99, Iter: 315, Loss: 0.71977
Epoch: 99, Iter: 316, Loss: 0.48588
Epoch: 99, Iter: 317, Loss: 0.42335
Epoch: 99, Iter: 318, Loss: 0.44606
Epoch: 99, Iter: 319, Loss: 0.42347
Epoch: 99, Iter: 320, Loss: 0.39549
Epoch: 99, Iter: 321, Loss: 0.43831
Epoch: 99, Iter: 322, Loss: 0.66199
Epoch: 99, Iter: 323, Loss: 0.30803
Epoch: 99, Iter: 324, Loss: 0.51772
Epoch: 99, Iter: 325, Loss: 0.49273
Epoch: 99, Iter: 326, Loss: 0.40635
Epoch: 99, Iter: 327, Loss: 0.54938
Epoch: 99, Iter: 328, Loss: 0.36019
Epoch: 99, Iter: 329, Loss: 0.42901
Epoch: 99, Iter: 330, Loss: 0.51389
Epoch: 99, Iter: 331, Loss: 0.64828
Epoch: 99, Iter: 332, Loss: 0.48830
Epoch: 99, Iter: 333, Loss: 0.53982
Epoch: 99, Iter: 334, Loss: 0.58016
Epoch: 99, Iter: 335, Loss: 0.44008
Epoch: 99, Iter: 336, Loss: 0.54616
Epoch: 99, Iter: 337, Loss: 

Epoch: 99, Iter: 547, Loss: 0.49673
Epoch: 99, Iter: 548, Loss: 0.54446
Epoch: 99, Iter: 549, Loss: 0.51663
Epoch: 99, Iter: 550, Loss: 0.45642
Epoch: 99, Iter: 551, Loss: 0.36577
Epoch: 99, Iter: 552, Loss: 0.68667
Epoch: 99, Iter: 553, Loss: 0.36764
Epoch: 99, Iter: 554, Loss: 0.41306
Epoch: 99, Iter: 555, Loss: 0.49892
Epoch: 99, Iter: 556, Loss: 0.60750
Epoch: 99, Iter: 557, Loss: 0.42122
Epoch: 99, Iter: 558, Loss: 0.55032
Epoch: 99, Iter: 559, Loss: 0.43788
Epoch: 99, Iter: 560, Loss: 0.65079
Epoch: 99, Iter: 561, Loss: 0.49813
Epoch: 99, Iter: 562, Loss: 0.38599
Epoch: 99, Iter: 563, Loss: 0.68855
Epoch: 99, Iter: 564, Loss: 0.51381
Epoch: 99, Iter: 565, Loss: 0.32528
Epoch: 99, Iter: 566, Loss: 0.37287
Epoch: 99, Iter: 567, Loss: 0.51443
Epoch: 99, Iter: 568, Loss: 0.69433
Epoch: 99, Iter: 569, Loss: 0.44572
Epoch: 99, Iter: 570, Loss: 0.53011
Epoch: 99, Iter: 571, Loss: 0.57443
Epoch: 99, Iter: 572, Loss: 0.43573
Epoch: 99, Iter: 573, Loss: 0.47031
Epoch: 99, Iter: 574, Loss: 

Epoch: 99, Iter: 803, Loss: 0.41429
Epoch: 99, Iter: 804, Loss: 0.78314
Epoch: 99, Iter: 805, Loss: 0.40629
Epoch: 99, Iter: 806, Loss: 0.67349
Epoch: 99, Iter: 807, Loss: 0.57720
Epoch: 99, Iter: 808, Loss: 0.56456
Epoch: 99, Iter: 809, Loss: 0.52259
Epoch: 99, Iter: 810, Loss: 0.40375
Epoch: 99, Iter: 811, Loss: 0.47414
Epoch: 99, Iter: 812, Loss: 0.42927
Epoch: 99, Iter: 813, Loss: 0.67171
Epoch: 99, Iter: 814, Loss: 0.46025
Epoch: 99, Iter: 815, Loss: 0.52611
Epoch: 99, Iter: 816, Loss: 0.64195
Epoch: 99, Iter: 817, Loss: 0.61235
Epoch: 99, Iter: 818, Loss: 0.53027
Epoch: 99, Iter: 819, Loss: 0.45591
Epoch: 99, Iter: 820, Loss: 0.55321
Epoch: 99, Iter: 821, Loss: 0.46283
Epoch: 99, Iter: 822, Loss: 0.41358
Epoch: 99, Iter: 823, Loss: 0.60412
Epoch: 99, Iter: 824, Loss: 0.73482
Epoch: 99, Iter: 825, Loss: 0.69526
Epoch: 99, Iter: 826, Loss: 0.51834
Epoch: 99, Iter: 827, Loss: 0.48460
Epoch: 99, Iter: 828, Loss: 0.48586
Epoch: 99, Iter: 829, Loss: 0.54181
Epoch: 99, Iter: 830, Loss: 